In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import time
import re
import shutil
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SequentialSampler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, jaccard_score, f1_score, homogeneity_completeness_v_measure, adjusted_rand_score
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, pairwise_distances
import numpy as np
from scipy.optimize import linear_sum_assignment
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -O Stsbenchmark.tar.gz http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
shutil.unpack_archive('./Stsbenchmark.tar.gz', extract_dir='./', format='gztar')

--2023-11-09 17:23:22--  http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.es (ixa2.si.ehu.es)... 158.227.106.100
Connecting to ixa2.si.ehu.es (ixa2.si.ehu.es)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz [following]
--2023-11-09 17:23:22--  http://ixa2.si.ehu.eus/stswiki/images/4/48/Stsbenchmark.tar.gz
Resolving ixa2.si.ehu.eus (ixa2.si.ehu.eus)... 158.227.106.100
Connecting to ixa2.si.ehu.eus (ixa2.si.ehu.eus)|158.227.106.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409630 (400K) [application/x-gzip]
Saving to: ‘Stsbenchmark.tar.gz’

Stsbenchmark.tar.gz 100%[===================>] 400.03K   411KB/s    in 1.0s    

2023-11-09 17:23:23 (411 KB/s) - ‘Stsbenchmark.tar.gz’ saved [409630/409630]



In [ ]:
def getSTSBenchmarkSents(filename='sts-train.csv', root='stsbenchmark/', encoding='utf-8'):
  f = open(root+filename, 'r', encoding=encoding)
  s1, s2, target = [], [], []
  for line in f:
    example = re.split(r'\t+', line)
    if len(example) > 7:
      example = example[:-2]
    s2.append(example[-1])
    s1.append(example[-2])
    target.append(float(example[-3]))
  print("{} samples: {}".format(filename, len(target)))
  return s1, s2, target

In [ ]:
s1_test,s2_test,target_test= getSTSBenchmarkSents(filename='sts-test.csv')

sts-test.csv samples: 1379


In [ ]:
BERT_PATH = "bert-base-uncased"
root_drive = '/content/drive/MyDrive/Tesis/STS_Benchmark/transformer_tunned_BERT/uncase_base/'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

Device: cpu


In [ ]:
#(num_sent, num_capa, num_cabezal)
CLS_vectors = torch.load(root_drive + BERT_PATH + '_all_CLS_outputs_vectors.pth', map_location=torch.device(device))

In [ ]:
CLS_vectors[(0,0)]['vectors'].shape

torch.Size([1, 768])

In [ ]:
layers = 12
heads = 12
num_sentences = list(CLS_vectors.keys())[-1][0]+1
CLS_vectors_list = []

for i in range(num_sentences):
  for j in range(layers):
    CLS_vectors_list.append(((i,j),CLS_vectors[(i,j)]['sequence'],CLS_vectors[(i,j)]['label'],CLS_vectors[(i,j)]['dimension'],CLS_vectors[(i,j)]['vectors']))

In [ ]:
CLS_vectors_list[0]

((0, 0),
 's1: A girl is styling her hair. s2: A girl is brushing her hair.',
 2.5,
 17,
 tensor([[ 1.3366e-02,  4.4970e-02, -2.1637e-01, -3.2843e-01,  2.6883e-01,
           3.4715e-02, -1.9444e-01, -1.1049e-02, -1.1818e-01, -3.9399e-01,
           3.0634e-02, -5.8748e-02,  1.3790e-01, -1.0854e-01, -1.1573e-01,
           4.3709e-01,  4.1830e-01,  2.6936e-01, -8.8575e-02, -5.6118e-01,
          -5.0836e-01, -3.3846e-01,  2.8790e-02, -1.9942e-01, -1.1913e-01,
          -2.5059e-01, -3.8456e-01,  4.5932e-02,  4.7195e-02, -1.2446e-01,
          -1.1421e-01, -1.6876e-01,  1.7933e-01,  1.6506e-01, -3.5994e-01,
           6.3980e-02, -3.9983e-01,  1.9067e-01, -2.0721e-01,  2.4676e-01,
           3.1398e-01, -1.7534e-01, -1.0620e-01,  1.6748e-01,  1.0974e-01,
           9.0651e-02, -1.5393e+00, -1.7551e-02,  5.5214e-01, -2.3338e-01,
           1.6909e-01,  5.9467e-02, -2.3357e-01,  8.8209e-01,  6.5783e-02,
          -1.4159e-01, -2.2780e-01,  1.0475e-01,  2.7281e-01, -2.2613e-01,
           

In [ ]:
CLS_vectors_list[0]

((0, 0),
 's1: A girl is styling her hair. s2: A girl is brushing her hair.',
 2.5,
 17,
 tensor([[ 1.3366e-02,  4.4970e-02, -2.1637e-01, -3.2843e-01,  2.6883e-01,
           3.4715e-02, -1.9444e-01, -1.1049e-02, -1.1818e-01, -3.9399e-01,
           3.0634e-02, -5.8748e-02,  1.3790e-01, -1.0854e-01, -1.1573e-01,
           4.3709e-01,  4.1830e-01,  2.6936e-01, -8.8575e-02, -5.6118e-01,
          -5.0836e-01, -3.3846e-01,  2.8790e-02, -1.9942e-01, -1.1913e-01,
          -2.5059e-01, -3.8456e-01,  4.5932e-02,  4.7195e-02, -1.2446e-01,
          -1.1421e-01, -1.6876e-01,  1.7933e-01,  1.6506e-01, -3.5994e-01,
           6.3980e-02, -3.9983e-01,  1.9067e-01, -2.0721e-01,  2.4676e-01,
           3.1398e-01, -1.7534e-01, -1.0620e-01,  1.6748e-01,  1.0974e-01,
           9.0651e-02, -1.5393e+00, -1.7551e-02,  5.5214e-01, -2.3338e-01,
           1.6909e-01,  5.9467e-02, -2.3357e-01,  8.8209e-01,  6.5783e-02,
          -1.4159e-01, -2.2780e-01,  1.0475e-01,  2.7281e-01, -2.2613e-01,
           

In [ ]:
CLS_vectors_list[0][4].shape

torch.Size([1, 768])

Dataloader

In [ ]:
# Crear un sampler secuencial
# attentions_list = dataset
# SequentialSampler does not perform any shuffling or random selection of items.
sampler = SequentialSampler(CLS_vectors_list)

# Definir el tamaño del lote
batch_size = 12 # always 12, because it is the number of attention layers, 12 layers for the same sentence

# Crear el DataLoader sin un BatchSampler
dataloader = DataLoader(CLS_vectors_list, batch_size=batch_size, sampler=sampler)

In [ ]:
len(dataloader) #1379 OK because there are 1379 sentences, 1 batch = 1 same sentence , 12 elements in the batch because there are 12 layers

1379

In [ ]:
sr = next(iter(dataloader))
sr[0] # ids (num_sent, num_layer)
sr[1] # similarity label
sr[2].shape
sr[3].shape
sr[4].shape #[batch_size, len_sentence, input_size] with attentions ([12,289,12])

torch.Size([12, 1, 768])

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def train_loop(model, iterator, optimizer, criterion, device=device, clip = 1.0):
    #Training loop
    model.train()
    loss_sum = 0
    seed = 42
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    #torch.cuda.set_device(0)

    for i, (_,_,_,_,input) in enumerate(iterator):
        optimizer.zero_grad()
        output, _ = model(input)
        loss = criterion(output, input)
        loss.backward()
        #prevent gradients from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        #Update params
        optimizer.step()
        loss_sum += loss.item()

    epoch_train_loss = loss_sum * batch_size / len(iterator)

    return epoch_train_loss

In [ ]:
# Extraer el vector latente fijo de cada elemento del batch
def extract_latent_vectors(model, dataloader, device, model_type='RNN'):
    model.eval()
    vector_representations = {}

    with torch.no_grad():
        for (id,s,label,dim,input) in dataloader:
            latent_vectors = []
            if model_type == 'RNN':
              _, (latent_representation, _) = model.encoder(input)
              latent = latent_representation.squeeze(0)
            else:
              latent_representation = model.encoder(input)
              latent = latent_representation.squeeze(0).squeeze(1)
            tuples = list(zip(id[0].tolist(), id[1].tolist()))
            #latent = latent_representation.squeeze(0)
            for i in range(latent.size(0)):
              latent_vectors.append(latent[i].numpy())
              vector_representations[tuples[i]] = { 'vector' : latent[i].numpy(), 'sequence': s, 'label': label, 'dimension':dim}

    return vector_representations

In [ ]:
class LinearAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(LinearAutoencoder, self).__init__()
        dims = [768, 512, 256, 128, 32, 8, 2]

        layer_dims= [i for i in dims if i > latent_dim ]
        layer_dims.append(latent_dim)

        encoder_layers = []
        decoder_layers = []

        for i in range(len(layer_dims) - 1):
            encoder_layers.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
            encoder_layers.append(nn.GELU())
            decoder_layers.insert(0, nn.Linear(layer_dims[i+1], layer_dims[i]))
            decoder_layers.insert(0, nn.GELU())

        self.encoder = nn.Sequential(*encoder_layers)
        self.decoder = nn.Sequential(*decoder_layers)

    def forward(self, x):
        # Codificación
        encoded = self.encoder(x)
        # Decodificación
        decoded = self.decoder(encoded)
        return decoded, encoded


In [ ]:
NUM_EPOCHS = 500
best_valid_loss = float('inf')
model_name = 'Autoencoder_Lineal_CLS'
train_loss_values = []
history = {"train": {"loss": []}}

input_size = 768  #due to 768 dimensión of BERT
hidden_size = 2  # size of fixed vector #laten dim
learning_rate = 0.00025 #0.00025 #0.0007

seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)


#model = AttentionLinearAutoencoder(input_size, hidden_size)
model = LinearAutoencoder(input_size, hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(NUM_EPOCHS):

    start_time = time.time()
    epoch_train_loss = train_loop(model,dataloader,optimizer,criterion,device)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    train_loss_values.append(epoch_train_loss)

    history["train"]["loss"].append(epoch_train_loss)

    print('-' * 80)
    #print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Epoch Time: {epoch_mins}m {epoch_secs}s | Train loss: {epoch_train_loss:.4f} | Train acc: {epoch_train_acc:.4f} | Dev loss: {epoch_dev_loss:.4f} | Dev acc: {epoch_dev_acc:.4f}')
    print(f'Epoch: {epoch+1:03}/{NUM_EPOCHS} | Epoch Time: {epoch_mins}m {epoch_secs}s | Train loss: {epoch_train_loss:.4f}')


--------------------------------------------------------------------------------
Epoch: 001/500 | Epoch Time: 0m 30s | Train loss: 1.2771
--------------------------------------------------------------------------------
Epoch: 002/500 | Epoch Time: 0m 25s | Train loss: 0.6560
--------------------------------------------------------------------------------
Epoch: 003/500 | Epoch Time: 0m 25s | Train loss: 0.5858
--------------------------------------------------------------------------------
Epoch: 004/500 | Epoch Time: 0m 26s | Train loss: 0.5597
--------------------------------------------------------------------------------
Epoch: 005/500 | Epoch Time: 0m 25s | Train loss: 0.5477
--------------------------------------------------------------------------------
Epoch: 006/500 | Epoch Time: 0m 24s | Train loss: 0.5420
--------------------------------------------------------------------------------
Epoch: 007/500 | Epoch Time: 0m 26s | Train loss: 0.5374
----------------------------------

In [ ]:
vector_representations = extract_latent_vectors(model, dataloader, device, model_type='Linear')

In [ ]:
vector_representations[(0,0)]['vector']

array([-0.07872151, -0.1469995 ], dtype=float32)

In [ ]:
# Guardar el diccionario en un archivo
torch.save(vector_representations, root_drive + BERT_PATH + str(hidden_size) + '_vector_representations_CLS_Linear2.pth')

In [ ]:
hidden_size = 2  # size of fixed vector
vector_representations = torch.load(root_drive + BERT_PATH + str(hidden_size) + '_vector_representations_CLS_Linear2.pth')

In [ ]:
len(vector_representations) #(num_sentence,layer)

16548

In [ ]:
def get_representations_per_layer(num_sentences, vector_representations, layers = 12):
  vectors_per_layer = {}
  labels = {}
  for l in range(layers):
    #vectors_per_layer[l] = np.array([vector_representations[(i,l)]['vector'].detach().numpy() for i in range(num_sentences)])
    vectors_per_layer[l] = np.array([vector_representations[(i,l)]['vector'] for i in range(num_sentences)])
  labels = { i: vector_representations[(i,0)]['label'][0].item() for i in range(num_sentences)}
  return vectors_per_layer, labels

In [ ]:
vectors_per_layer, labels = get_representations_per_layer(num_sentences, vector_representations)

In [ ]:
# Guardar el diccionario en un archivo
torch.save(vectors_per_layer, root_drive + BERT_PATH + str(hidden_size) + '_vectors_per_layer_CLS_LSTM.pth')